### Trabalho 1 - CG
João Pedro Duarte Nunes 12542460
Raphael Leveque 12542522

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

INFINITO = math.inf

### Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL para vertex shader

In [3]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 mat_transform;        
        
        void main(){
            gl_Position = mat_transform * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

In [4]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [9]:
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [10]:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo-aula da Aula 11 - Parte 1, nós descrevemos o funcionamento desse formato.

In [11]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    for line in open(filename, "r"): 
        if line.startswith('#'): continue 
        values = line.split()
        if not values: continue

        if values[0] == 'v':
            x = float(values[1])
            y = float(values[2])
            z = float(values[3])
            vertices.append((x, y, z))

        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model



### Carregando texturas: LINEAR/NEAREST

In [12]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_linear_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

In [13]:
def load_texture_nearest_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_NEAREST)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_NEAREST)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

Obter os vértices do modelo. Deveremos, depois, normalizá-los

In [14]:
vertices_list = []    
textures_coord_list = []

### Processamento dos objetos
Agora, vamos processar cada um dos 5 objetos:

In [15]:
## modelo madara
modelo = load_model_from_file('Objects/Madara.obj')

print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Vertice final:',len(vertices_list))


load_texture_linear_from_file(0,'Objects/Madara.png')
load_texture_nearest_from_file(5, 'Objects/Madara.png')


FileNotFoundError: [Errno 2] No such file or directory: 'Objects/Madara.obj'

In [ ]:

modelo = load_model_from_file('Objects/monstro.obj')

print('Vertice inicial:', len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append(modelo['vertices'][vertice_id - 1])
    for texture_id in face[1]:
        textures_coord_list.append(modelo['texture'][texture_id - 1])

        
print('Vertice final:', len(vertices_list))

load_texture_linear_from_file(1,'Objects/monstro.jpg')
load_texture_nearest_from_file(6, 'Objects/monstro.jpg')

Vertice inicial: 68968
Vertice final: 75076


In [ ]:
## modelo planta
modelo = load_model_from_file('Objects/plant.obj')

print('Processando modelo plant.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo plant.obj. Vertice final:',len(vertices_list))

load_texture_linear_from_file(2,'Objects/plant3.jpg')
load_texture_nearest_from_file(7, 'Objects/plant3.jpg')

Processando modelo plant.obj. Vertice inicial: 75076
Processando modelo plant.obj. Vertice final: 166756


In [ ]:
## modelo cafe
modelo = load_model_from_file('Objects/coffee.obj')

print('Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Vertice final:',len(vertices_list))

load_texture_linear_from_file(3,'Objects/coffee.png')
load_texture_nearest_from_file(8, 'Objects/coffee.png')

Vertice inicial: 166756
Vertice final: 205876


In [ ]:
## modelo arma
modelo = load_model_from_file('Objects/Gun.obj')

vertices_list1 = []

print('Processando modelo gun.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list1.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo gun.obj. Vertice final:',len(vertices_list))

load_texture_linear_from_file(4,'Objects/Gun.png')
load_texture_nearest_from_file(9, 'Objects/Gun.png')

Processando modelo gun.obj. Vertice inicial: 205876
Processando modelo gun.obj. Vertice final: 205876


In [ ]:
def calculate_min_max_coordinates(vertices):
    """
    Calcula as coordenadas mínimas e máximas para cada dimensão (i, j, k) em uma lista de vértices.

    Parâmetros:
    - vertices (list): Uma lista de vértices, onde cada vértice é representado por uma tupla ou lista de três valores (i, j, k).

    Retorna:
    - list: Uma lista contendo os valores mínimos e máximos para cada dimensão na ordem [min_i, max_i, min_j, max_j, min_k, max_k].
    """
    min_i = math.inf
    max_i = -math.inf
    min_j = math.inf
    max_j = -math.inf
    min_k = math.inf
    max_k = -math.inf

    for index in range(len(vertices)):
        min_i = min(min_i, vertices[index][0])
        max_i = max(max_i, vertices[index][0])
        min_j = min(min_j, vertices[index][1])
        max_j = max(max_j, vertices[index][1])
        min_k = min(min_k, vertices[index][2])
        max_k = max(max_k, vertices[index][2])

    min_max_coordinates = [min_i, max_i, min_j, max_j, min_k, max_k]
    return min_max_coordinates

In [ ]:
def normalizar_vertices(vertices):
    """
    Normaliza as coordenadas dos vértices usando a normalização z-score.

    Parâmetros:
    - vertices (list): Uma lista de vértices, onde cada vértice é representado por uma tupla ou lista de três valores (x, y, z).

    Retorna:
    - list: Uma lista de vértices normalizados usando a normalização z-score.
    """
    # Extrai as coordenadas em listas separadas para facilitar o cálculo.
    xs, ys, zs = zip(*vertices)

    # Calcula a média e o desvio padrão de cada dimensão.
    mean_x = sum(xs) / len(xs)
    mean_y = sum(ys) / len(ys)
    mean_z = sum(zs) / len(zs)
    std_x = (sum((x - mean_x) ** 2 for x in xs) / len(xs)) ** 0.5
    std_y = (sum((y - mean_y) ** 2 for y in ys) / len(ys)) ** 0.5
    std_z = (sum((z - mean_z) ** 2 for z in zs) / len(zs)) ** 0.5

    # Aplica a normalização z-score aos vértices.
    vertices_normalizados = [
        ((x - mean_x) / std_x, (y - mean_y) / std_y, (z - mean_z) / std_z)
        for x, y, z in vertices
    ]

    return vertices_normalizados


# Exemplo de uso:
vertices = [(1, 2, 3), (4, 5, 6), (7, 8, 9)]
normalized_vertices = normalizar_vertices(vertices)
print(normalized_vertices)

[(-1.2247448713915892, -1.2247448713915892, -1.2247448713915892), (0.0, 0.0, 0.0), (1.2247448713915892, 1.2247448713915892, 1.2247448713915892)]


In [ ]:
# normalização de todos os vertices que serão enviados para a cpu 
vertices_list1 = normalizar_vertices(vertices_list1) 
normalized_vertices = normalizar_vertices(vertices_list)
normalized_vertices = normalized_vertices + vertices_list1

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

In [ ]:
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

In [ ]:
vertices = np.zeros(len(normalized_vertices), [("position", np.float32, 3)])
vertices['position'] = normalized_vertices


glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [ ]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


### Agora, definimos todos os eventos:
Modificam escala, rotação, translação etc


In [ ]:
class CharacterOptions:
    MADARA = 1
    CAIXA = 2
    PLANTA = 3
    CAFE = 4
    ARMA = 5

class Keys:
    num1 = 49
    num2 = 50
    num3 = 51
    num4 = 52
    num5 = 53
    A = 65
    D = 68
    P = 80
    S = 83
    V = 86
    W = 87
    X = 88
    Z = 90
    left = 263
    right = 262
    up = 265
    down = 264

polygonal_mode = False
nearest_mode = False
fator_escala = 0.0
object_x = 0.0
object_y = 0.0
rotacao_x = 0.0
rotacao_y = 0.0

In [ ]:


selected_character = CharacterOptions.MADARA

def handle_scale(key, action):
    global fator_escala
    if key == Keys.Z and action == glfw.PRESS:
        fator_escala += 0.05
    elif key == Keys.X and action == glfw.PRESS:
        fator_escala -= 0.05

def handle_character_selection(key, action):
    global selected_character
    if key in [Keys.num1, Keys.num2, Keys.num3, Keys.num4, Keys.num5] and action == glfw.PRESS:
        selected_character = {
            Keys.num1: CharacterOptions.MADARA,
            Keys.num2: CharacterOptions.CAIXA,
            Keys.num3: CharacterOptions.PLANTA,
            Keys.num4: CharacterOptions.CAFE,
            Keys.num5: CharacterOptions.ARMA
        }[key]

def toggle_polygonal_mode(key, action):
    global polygonal_mode
    if key == Keys.P and action == glfw.PRESS:
        polygonal_mode = not polygonal_mode

def toggle_nearest_mode(key, action):
    global nearest_mode
    if key == Keys.V and action == glfw.PRESS:
        nearest_mode = not nearest_mode

def handle_object_movement(key, action):
    global object_x, object_y
    if action == glfw.PRESS:
        if key == Keys.A:
            object_x -= 0.1
        elif key == Keys.D:
            object_x += 0.1
        elif key == Keys.S:
            object_y -= 0.1
        elif key == Keys.W:
            object_y += 0.1

def handle_rotation(key, action):
    global rotacao_x, rotacao_y
    if action == glfw.PRESS:
        if key == Keys.left:
            rotacao_y -= 0.2
        elif key == Keys.right:
            rotacao_y += 0.2
        elif key == Keys.up:
            rotacao_x -= 0.2
        elif key == Keys.down:
            rotacao_x += 0.2

def pressing_keys(window, key, scancode, action, mods):
    handle_scale(key, action)
    handle_character_selection(key, action)
    toggle_polygonal_mode(key, action)
    toggle_nearest_mode(key, action)
    handle_object_movement(key, action)
    handle_rotation(key, action)

# Define a função de callback de tecla para a janela.
glfw.set_key_callback(window, pressing_keys)


### Nesse momento, nós exibimos a janela!


In [ ]:
glfw.show_window(window)

### Desenhando os modelos

In [ ]:
def draw_madara():
    if nearest_mode==True:
        glBindTexture(GL_TEXTURE_2D, 5)
    if nearest_mode==False:
        glBindTexture(GL_TEXTURE_2D, 0)

    glDrawArrays(GL_QUADS, 0,68968) 

def draw_caixa():
    if nearest_mode:
        glBindTexture(GL_TEXTURE_2D, 6)
    else:
        glBindTexture(GL_TEXTURE_2D, 1)
        
    glDrawArrays(GL_QUADS, 68968, 75076 - 68968) 
    

def draw_planta():
    if nearest_mode==True:
        glBindTexture(GL_TEXTURE_2D, 7)
    if nearest_mode==False:
        glBindTexture(GL_TEXTURE_2D, 2)

    glDrawArrays(GL_QUADS, 75076,166756 - 75076) 

def draw_cafe():
    if nearest_mode==True:
        glBindTexture(GL_TEXTURE_2D, 8)
    if nearest_mode==False:
        glBindTexture(GL_TEXTURE_2D, 3)

    glDrawArrays(GL_QUADS, 166756,205876 - 166756) 


def draw_arma():
    if nearest_mode==True:
        glBindTexture(GL_TEXTURE_2D, 9)
    if nearest_mode==False:
        glBindTexture(GL_TEXTURE_2D, 4)

    glDrawArrays(GL_QUADS, 205876,205876) 


### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

# Classe MatrixModel

A classe `MatrixModel` representa uma matriz de transformação 3D para um modelo em uma aplicação gráfica. Inclui funcionalidades para translação, rotação, dimensionamento e transformação do modelo.

## Atributos

- `fator_escala`: Fator padrão de escala definido como 0.6.
- `translacao_x`, `translacao_y`, `translacao_z`: Valores iniciais de translação ao longo dos eixos X, Y e Z, respectivamente.
- `rotacao_x`, `rotacao_y`: Ângulos iniciais de rotação em torno dos eixos X e Y, respectivamente.

## Inicialização de Matrizes

- `matriz_transformacao`: Matriz identidade inicializada com shape (4, 4) e tipo float32.
- `matriz_rotacao_x`, `matriz_rotacao_y`: Matrizes identidade para rotações nos eixos X e Y.
- `matriz_escala`: Matriz identidade para dimensionamento.
- `matriz_translacao`: Matriz identidade para translação.
- `matriz_translacao_centro`: Matriz identidade para translação baseada no centro de coordenadas.

In [ ]:
import numpy as np
import math

class MatrixModel:
    fator_escala = 0.3
    translacao_x = 0.0
    translacao_v = 0.0
    translacao_z = 0.0
    rotacao_x = 0.0 
    rotacao_y = 0.0 
    def __init__(self):
        self.matriz_transformacao = np.identity(4, dtype=np.float32)
        self.matriz_rotacao_x = np.identity(4, dtype=np.float32)
        self.matriz_rotacao_y = np.identity(4, dtype=np.float32)
        self.matriz_escala = np.identity(4, dtype=np.float32)
        self.matriz_translacao = np.identity(4, dtype=np.float32)
        self.matriz_translacao_centro = np.identity(4, dtype=np.float32)

    @property
    def transform_matrix(self):
        return self.matriz_transformacao

    def multiplica_matriz(self, a, b):
        A = a.reshape(4, 4)
        B = b.reshape(4, 4)
        return np.dot(A, B).reshape(1, 16)

    def criar_matriz_escala(self):
        self.matriz_escala = np.array([self.fator_escala, 0.0, 0.0, 0.0,
                                0.0, self.fator_escala, 0.0, 0.0,
                                0.0, 0.0, self.fator_escala, 0.0,
                                0.0, 0.0, 0.0, 1.0], np.float32)

    def criar_matriz_translacao(self):
        self.matriz_translacao = np.array([1.0, 0.0, 0.0,0.0,
                                        0.0, 1.0, 0.0, 0.0,
                                        0.0, 0.0, 1.0, 0.0,
                                     self.translacao_x,self.translacao_v, self.translacao_z, 1.0], np.float32)
    def criar_matriz_rotacao(self):
        """
        Define as matrizes de rotação em torno dos eixos X e Y com base nos ângulos de rotação.

        Atributos:
        - self.rotacao_x: Ângulo de rotação em torno do eixo X em radianos.
        - self.rotacao_y: Ângulo de rotação em torno do eixo Y em radianos.

        As matrizes resultantes são armazenadas nos atributos self.matriz_rotacao_x e self.matriz_rotacao_y.
        """
        cos_x, sin_x = np.cos(self.rotacao_x), np.sin(self.rotacao_x)
        cos_y, sin_y = np.cos(self.rotacao_y), np.sin(self.rotacao_y)

        self.matriz_rotacao_x = np.array([1.0, 0.0, 0.0, 0.0,
                                        0.0, cos_x, -sin_x, 0.0,
                                        0.0, sin_x, cos_x, 0.0,
                                        0.0, 0.0, 0.0, 1.0], dtype=np.float32)

        self.matriz_rotacao_y = np.array([cos_y, 0.0, sin_y, 0.0,
                                        0.0, 1.0, 0.0, 0.0,
                                        -sin_y, 0.0, cos_y, 0.0,
                                        0.0, 0.0, 0.0, 1.0], dtype=np.float32)
    
    def criar_matriz_centro_translacao(self):
        self.matriz_translacao_centro = np.array([1.0, 0.0, 0.0,0.0,
                                    0.0, 1.0, 0.0, 0.0,
                                    0.0, 0.0, 1.0, 0.0,
                                    - self.translacao_x, - self.translacao_v, - self.translacao_z, 1.0], np.float32)


    def alterar_matriz_transformacao(self):
        self.criar_matriz_rotacao()
        self.criar_matriz_escala()
        self.criar_matriz_translacao()
        self.criar_matriz_centro_translacao()

        transform_temp = self.multiplica_matriz(self.matriz_translacao, self.matriz_translacao_centro)
        transform_temp = self.multiplica_matriz(self.matriz_rotacao_x, transform_temp)
        transform_temp = self.multiplica_matriz(self.matriz_rotacao_y, transform_temp)
        transform_temp = self.multiplica_matriz(self.matriz_escala, transform_temp)
        transform_temp = self.multiplica_matriz(self.matriz_translacao, transform_temp)

        self.matriz_transformacao = transform_temp



In [ ]:
"""
Configura um ambiente de renderização 3D usando OpenGL. Ativa o teste de profundidade e itera sobre modelos de personagens, aplicando transformações e renderizando.

Características:
- Usa uma lista de funções de desenho para personagens específicos.
- Cria instâncias da classe MatrixModel para cada personagem.
- Atualiza o personagem selecionado (inicialmente "caixa") com transformações como rotação, translação e escala.
- Utiliza OpenGL para renderizar os personagens na janela.
"""
glEnable(GL_DEPTH_TEST)

import numpy as np
import math

character_functions = [draw_madara, draw_caixa, draw_planta, draw_cafe, draw_arma]
caixa_model = MatrixModel()
madara_model = MatrixModel()
planta_model = MatrixModel()
arma_model = MatrixModel()
cafe_model = MatrixModel()
character_models = [madara_model, caixa_model, planta_model, arma_model, cafe_model]
selected_character = CharacterOptions.MADARA

running = not glfw.window_should_close(window)

while running:
    glfw.poll_events()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    glPolygonMode(GL_FRONT_AND_BACK, GL_LINE) if polygonal_mode else glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)

    for index, character_model in enumerate(character_models):
        if (index + 1) == selected_character:
            character_model.rotacao_x += rotacao_x
            rotacao_x = 0
            character_model.rotacao_y += rotacao_y
            rotacao_y = 0
            translacao_x, translacao_v = character_model.translacao_x, character_model.translacao_v
            character_model.translacao_x += object_x
            character_model.translacao_v += object_y
            object_x, object_y = 0, 0
            character_model.fator_escala += fator_escala
            fator_escala = 0.0
            character_model.alterar_matriz_transformacao()
            transform = character_model.transform_matrix
            loc_mat_transform = glGetUniformLocation(program, "mat_transform")
            glUniformMatrix4fv(loc_mat_transform, 1, GL_FALSE, transform)
            character_functions[index]()

    glfw.swap_buffers(window)
    running = not glfw.window_should_close(window)

glfw.terminate()


1   HIToolbox                           0x00000001b5d785c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x00000001b5d78188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x00000001b5ce58bc _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1184
4   HIToolbox                           0x00000001b5d78004 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 180
5   AppKit                              0x00000001afb67d30 -[NSApplication _setPresentationOptions:instance:flags:] + 956
6   AppKit                              0x00000001af9fd93c -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 404
7   CoreFoundation                      0x00000001ac61a560 __CFNOTIFICATIONCENTER_IS_CALLING_OUT_TO_AN_OBSERVER__ + 148
8   CoreFoundation                      0x00000001ac6b8044 ___CFXRegistrationPost_block_invoke + 88
9   CoreFoundation                      0x00000001ac6b7f8c _CFXRe

KeyboardInterrupt: 